# Goodbooks Exercise
### Load the Data

In [1]:
import pandas as pd

path = 'https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv'
book_df = pd.read_csv(path)


### Check the Data for import problems

In [2]:
import string

# Here I am making sure that there are no null values in the column I will be checking
book_df['original_title'] = book_df.original_title.fillna('No Title')

# Here I am checking to see if a title has characters that are not in the english alphabet
# I am checking the string with a regex expression for the check
book_df[(~book_df.original_title.str.contains("|".join(string.ascii_letters)))].head()

# Based on the result of this check, I don't need to do anything special on data import

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
78,79,1381,1381,3356006,1703,143039954,9.780143e+12,"Homer, Robert Fagles, E.V. Rieu, Frédéric Mugl...",-720.0,Ὀδύσσεια,...,670326,710757,8101,29703,65629,183082,224120,208223,https://images.gr-assets.com/books/1390173285m...,https://images.gr-assets.com/books/1390173285s...
171,172,15823480,15823480,2507928,1492,345803922,9.780346e+12,"Leo Tolstoy, Louise Maude, Leo Tolstoj, Aylmer...",1877.0,Анна Каренина,...,297472,472796,18064,11738,26945,88365,158179,187569,https://images.gr-assets.com/books/1352422904m...,https://images.gr-assets.com/books/1352422904s...
176,177,7144,7144,3393917,1714,143058142,9.780143e+12,"Fyodor Dostoyevsky, David McDuff",1866.0,Преступление и наказание,...,380903,444675,12605,9477,20078,64050,137104,213966,https://images.gr-assets.com/books/1382846449m...,https://images.gr-assets.com/books/1382846449s...
294,295,10644930,10644930,15553789,145,1451627289,9.781452e+12,Stephen King,2011.0,11/22/63,...,258464,303057,30656,2927,7649,35500,109392,147589,https://images.gr-assets.com/books/1327876792m...,https://images.gr-assets.com/books/1327876792s...
340,341,1371,1371,3293141,1726,140275363,9.780140e+12,"Homer, Robert Fagles, Frédéric Mugler, Bernard...",-750.0,Ἰλιάς,...,241088,273565,4763,7701,20845,68844,89384,86791,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


### Decide how to handle the empty values in columns
 - I will be replacing these nulls with a "Not Provided" label because the books could still be useful without that data.

In [3]:
book_df['isbn'] = book_df.isbn.fillna('Not Provided')
book_df['isbn13'] = book_df.isbn13.fillna('Not Provided')
book_df['language_code'] = book_df.language_code.fillna('Not Provided')

### Format ISBN 13 to match specifications

In [4]:
# This function checks for what numbers will accurately fit the isbn requirements
def check_sum(num, end):
    sum_check = 0
    for i, char in enumerate(num[:-end]):
        if i % 2 == 0:
            sum_check += int(char) * 1
        else:
            sum_check += int(char) * 3
    check_digit = str(int(round(sum_check + 5.1, -1) - sum_check))
    if end == 2:
        new_isbn = num[:-2] + check_digit + '0'
    else:
        new_isbn = num[:-1] + check_digit
    return new_isbn

# This function checks whether to try formatting an isbn and how to do so.
def format_isbn(isbn, isbn13):
    
    # If the isbn was not provided, skip it.
    if isbn13 == 'Not Provided':
        return isbn13, 'skipped'
    
    # Set a default fall through return value
    method = 'not_calculated'
    new_isbn = 'not_determined'
    
    # convert both isbn versions to strings
    isbn = str(isbn)
    isbn13 = str(int(isbn13))

    # Checks whether just the last number doesn't match or if the last two have a problem
    if isbn[:-1] in isbn13:      
        new_isbn = check_sum(isbn13, 1)
        method = "single"
    else:
        # If the last two don't match try to mutate based on the second from last, and then the last
        if isbn[:-2] in isbn13:
            new_isbn = check_sum(isbn13, 2)
            new_isbn = check_sum(new_isbn, 1)
            method = 'double'
            
    s = new_isbn
    new_num = f'{s[:3]}-{s[3:4]}-{s[4:9]}-{s[9:12]}-{s[12:]}'
    return new_num, method
    
book_df[['new_isbn13','isbn_mutation']] = book_df.apply(lambda row: format_isbn(row.isbn, row.isbn13), axis = 1, result_type = 'expand')
book_df[['isbn', 'isbn13', 'new_isbn13','title','isbn_mutation']]

,isbn,isbn13,new_isbn13,title,isbn_mutation
0,439023483,9780439023480.0,978-0-43902-348-1,"The Hunger Games (The Hunger Games, #1)",single
1,439554934,9780439554930.0,978-0-43955-493-10,Harry Potter and the Sorcerer's Stone (Harry P...,single
2,316015849,9780316015840.0,978-0-31601-584-4,"Twilight (Twilight, #1)",single
3,61120081,9780061120080.0,978-0-06112-008-4,To Kill a Mockingbird,single
4,743273567,9780743273560.0,978-0-74327-356-5,The Great Gatsby,single
...,...,...,...,...,...
9995,441019455,9780441019460.0,978-0-44101-943-4,"Bayou Moon (The Edge, #2)",double
9996,067973371X,9780679733710.0,978-0-67973-371-3,"Means of Ascent (The Years of Lyndon Johnson, #2)",single
9997,039330762X,9780393307630.0,978-0-39330-763-4,The Mauritius Command,double
9998,61711527,9780061711530.0,978-0-06171-153-4,Cinderella Ate My Daughter: Dispatches from th...,double


### Add a column to show where original title and title differ.

In [5]:
book_df['title_mismatch'] = book_df.apply(lambda row: 'match' if row.title == row.original_title else 'difference', axis = 1)
book_df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,new_isbn13,isbn_mutation,title_mismatch
0,1,2767052,2767052,2792775,272,439023483,9780439023480.0,Suzanne Collins,2008.0,The Hunger Games,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,978-0-43902-348-1,single,difference
1,2,3,3,4640799,491,439554934,9780439554930.0,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,978-0-43955-493-10,single,difference
2,3,41865,41865,3212258,226,316015849,9780316015840.0,Stephenie Meyer,2005.0,Twilight,...,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,978-0-31601-584-4,single,difference
3,4,2657,2657,3275794,487,61120081,9780061120080.0,Harper Lee,1960.0,To Kill a Mockingbird,...,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,978-0-06112-008-4,single,match
4,5,4671,4671,245494,1356,743273567,9780743273560.0,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,978-0-74327-356-5,single,match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9780441019460.0,Ilona Andrews,2010.0,Bayou Moon,...,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,978-0-44101-943-4,double,difference
9996,9997,208324,208324,1084709,19,067973371X,9780679733710.0,Robert A. Caro,1990.0,Means of Ascent,...,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,978-0-67973-371-3,single,difference
9997,9998,77431,77431,2393986,60,039330762X,9780393307630.0,Patrick O'Brian,1977.0,The Mauritius Command,...,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...,978-0-39330-763-4,double,match
9998,9999,8565083,8565083,13433613,7,61711527,9780061711530.0,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...,978-0-06171-153-4,double,match


### Break up authors into separate columns

In [6]:
# I am creating a new df with an index of the unique book_id
author_df = book_df.set_index('book_id').authors.str.split(",", expand = True).fillna("-")

# I am renaming the columns
author_df.columns = ['Author ' + str(i) for i in range(1,len(author_df.columns) + 1)]

# I am moving the index back to a column
author_df.reset_index(inplace = True)

author_separated_df = pd.merge(author_df, book_df, left_on = 'book_id', right_on = 'book_id')
author_separated_df.head()

,book_id,Author 1,Author 2,Author 3,Author 4,Author 5,Author 6,Author 7,Author 8,Author 9,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,new_isbn13,isbn_mutation,title_mismatch
0,1,Suzanne Collins,-,-,-,-,-,-,-,-,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,978-0-43902-348-1,single,difference
1,2,J.K. Rowling,Mary GrandPré,-,-,-,-,-,-,-,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,978-0-43955-493-10,single,difference
2,3,Stephenie Meyer,-,-,-,-,-,-,-,-,...,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,978-0-31601-584-4,single,difference
3,4,Harper Lee,-,-,-,-,-,-,-,-,...,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,978-0-06112-008-4,single,match
4,5,F. Scott Fitzgerald,-,-,-,-,-,-,-,-,...,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,978-0-74327-356-5,single,match


### Store each publication by its publication year by century in its own worksheet.

In [7]:
# Not yet checked for accuracy, but I think it is fine.

year_groups = [(1700,1800), (1800,1900), (1900,2000), (2000,2100)]

with pd.ExcelWriter('book_data_by_year.xlsx') as new_file:
    books_written = 0
    for year_group in year_groups:
        start_year = year_group[0]
        end_year = year_group[1]
        new_sheet_name = f'{start_year}-{end_year-1}'
        
        subset = author_separated_df[
            (author_separated_df.original_publication_year.isin(range(start_year, end_year, 1)))
        ]
        subset.to_excel(new_file, sheet_name = new_sheet_name)
        print(len(subset))
        books_written += subset.shape[0]
        print(books_written)
        
        author_separated_df = author_separated_df[
            (~author_separated_df.original_publication_year.isin(range(start_year, end_year, 1)))
        ]
    
    author_separated_df.to_excel(new_file, sheet_name = 'Other Years')
    books_written += author_separated_df.shape[0]

print(f'Books Written to file: {books_written}')

26
26
254
280
3412
3692
6188
9880
Books Written to file: 10000


In [8]:
book_df.shape[0]

10000